In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from win32cryptcon import CMSG_CONTENT_ENCRYPT_PAD_ENCODED_LEN_FLAG
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



### Experiment 3: Train XGBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Track Experiments Using MLFlow

In [7]:
models = [
    (
        'LogisticRegression',
        LogisticRegression(C=1, solver='liblinear'),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        'RandomForestClassifier',
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        'XGBClassifier',
        XGBClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test),
    )
]

In [8]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train, y_train = train_set
    X_test, y_test = test_set
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [10]:
reports[0]

{'0': {'precision': 0.9454545454545454,
  'recall': 0.9629629629629629,
  'f1-score': 0.9541284403669725,
  'support': 270.0},
 '1': {'precision': 0.6,
  'recall': 0.5,
  'f1-score': 0.5454545454545454,
  'support': 30.0},
 'accuracy': 0.9166666666666666,
 'macro avg': {'precision': 0.7727272727272727,
  'recall': 0.7314814814814814,
  'f1-score': 0.749791492910759,
  'support': 300.0},
 'weighted avg': {'precision': 0.9109090909090909,
  'recall': 0.9166666666666666,
  'f1-score': 0.91326105087573,
  'support': 300.0}}

In [14]:
import mlflow
import sklearn
import xgboost

mlflow.set_experiment('Anomaly Detection')
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

for i, element in enumerate(models):
    name = element[0]
    model = element[1]
    params = dict()
    params['accuracy'] = reports[i]['accuracy']
    params['recall_class_0'] = reports[i]['0']['recall']
    params['recall_class_1'] = reports[i]['1']['recall']
    params['f1_score_macro'] = reports[i]['macro avg']['f1-score']
    params['f1_score_weighted'] = reports[i]['weighted avg']['f1-score']

    with mlflow.start_run(run_name=name):

        mlflow.log_metrics(params)

        mlflow.log_param('model_name', name)

        if "XGB" in name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')